In [13]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_recall_fscore_support
import random
import matplotlib.pyplot as plt
import itertools

# k-nearest neighbors

This dataset was obtained from https://archive.ics.uci.edu/ml/datasets/Heart+Disease (this is a great resource for datasets to try machine learning on). It has data on patients that are and are not diagnosed with heart disease.

The attributes are:
* age: age in years 
* sex: sex (1 = male; 0 = female) 
* cp: chest pain type 
 * -- Value 1: typical angina 
 * -- Value 2: atypical angina 
 * -- Value 3: non-anginal pain 
 * -- Value 4: asymptomatic 
* trestbps: resting blood pressure (in mm Hg on admission to the hospital) 
* chol: serum cholestoral in mg/dl 
* fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) 
* restecg: resting electrocardiographic results 
 * -- Value 0: normal 
 * -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) 
 * -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria 
* thalach: maximum heart rate achieved 
* exang: exercise induced angina (1 = yes; 0 = no) 
* oldpeak = ST depression induced by exercise relative to rest 
* slope: the slope of the peak exercise ST segment 
 * -- Value 1: upsloping 
 * -- Value 2: flat 
 * -- Value 3: downsloping 
* ca: number of major vessels (0-3) colored by flourosopy 
* thal: 3 = normal; 6 = fixed defect; 7 = reversable defect 
* num: diagnosis of heart disease (angiographic disease status) 
 * -- Value 0: absence.
 * -- Value 1,2,3,4: presence of heart disease


# Explore the data

Read in the data, modify the dependent variable name and plot a histogram of the ages of patients, both healthy and those with heart disease.

In [14]:
df = pd.read_csv('cleveland.csv')

# Collapse all values 1-4 into a single value so that "num" is boolean
df = df.rename({'num':'disease'}, axis=1)
df['disease'] = df.disease.apply(lambda x: min(x, 1))
df.drop(df[df['thal'] == '?'].index, inplace=True)
display(df.head(5))

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,disease
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


# multiple dimensions


In [15]:
def knnPredict(data, attributes, numOfNeighbors, target='disease'):
    nn = NearestNeighbors(n_neighbors=numOfNeighbors, metric='euclidean', algorithm='auto')
    
    # Standardize
    newAttributes = []
    for attribute in attributes:
        standard = data[attribute]-data[attribute].mean()/data[attribute].std()
        standard.name = standard.name+'_s'
        newAttributes.append(standard.name)
        data = pd.concat([data,standard],axis=1)
        
    # Build underlying structure with standardized data   
    X = data[newAttributes].values
    y = data[target].values
    fit = nn.fit(X)
    
    # Choose a random patient
    i = random.randint(0,len(X)-1)
    patientX = X[i]
    patienty = y[i]
    # display('Our patient',data.iloc[i])
    
    # Find the k nearest neighbors, not including self
    distances, indices = fit.kneighbors([patientX],numOfNeighbors+1)
    nbrs = data.iloc[indices[0]]
    # Delete self from data
    nbrs = nbrs.drop(data.iloc[i].name)
    # display(nbrs)
    
    # Count the number of neighbors that have target
    have = nbrs[nbrs[target] == 0].count()[target]
    # Count the number of neighbors that DON'T have target
    dontHave = nbrs[nbrs[target] == 1].count()[target]
    # print('Have: {}\ndontHave: {}'.format(have, dontHave))
    
    # Predict that our random patient is like the majority of its neighbors
    predict = 0 if (dontHave > have) else 1
    # According to the records, did our patient have the target
    actual = 0 if (patienty == 0) else 1
    success = predict == actual
    # print('Sucess:',success)
    return success, patienty == 0

In [16]:
def testMeSenpai(data, attributes, numOfNeighbors, numOfTests=1, target='disease'):
    y_pred = []
    y_true = []
    for test in range(numOfTests):
        yPred, yAct = knnPredict(data, attributes, numOfNeighbors)
        y_pred.append(yPred)
        y_true.append(yAct)
    return y_pred ,y_true

In [17]:
def crossValidate(data, attributes, numOfNeighbors, numOfTests, target='disease', level = 10):
    folds = []
    for i in range(level):
        folds.append(data.sample(frac = 1/level))
        
    # Train missing one fold
    for i in range(level-1):
        temp = pd.concat([data,folds[i]])
        temp = temp.drop_duplicates(keep=False)
        
        y_pred,y_true = testMeSenpai(temp, attributes, numOfNeighbors, numOfTests, target='disease')
        (p,r,f,s) = precision_recall_fscore_support(y_pred,y_true,zero_division=0)
        print(p,r,f,s,'\n')
        
df1 = df.copy()
k = 5
numOfTests = 5
attributes = ['age','trestbps']

crossValidate(df1,attributes,k,numOfTests)
# y_pred, y_true = testMeSenpai(df1,attributes,k,numOfTests)
# (p,r,f,s) = precision_recall_fscore_support(y_pred,y_true)
# display(p,r,f,s)

[0.33333333 0.        ] [0.33333333 0.        ] [0.33333333 0.        ] [3 2] 

[0.33333333 0.5       ] [0.5        0.33333333] [0.4 0.4] [2 3] 

[0.5 1. ] [1.         0.33333333] [0.66666667 0.5       ] [2 3] 

[0.6 0. ] [1. 0.] [0.75 0.  ] [3 2] 

[1.         0.66666667] [0.66666667 1.        ] [0.8 0.8] [3 2] 

[0.75 1.  ] [1.  0.5] [0.85714286 0.66666667] [3 2] 

[0.5        0.66666667] [0.5        0.66666667] [0.5        0.66666667] [2 3] 

[0.33333333 0.        ] [0.33333333 0.        ] [0.33333333 0.        ] [3 2] 

[0. 1.] [0.  0.6] [0.   0.75] [0 5] 



In [18]:
def testAgainstTest(testData,attributes,k):
    y_pred, y_true = testMeSenpai(testData,attributes,k)
    (p,r,f,s) = precision_recall_fscore_support(y_pred,y_true)
    return p,r,f,s

The model is the value of k (nearest neigbors) and set of attributes

## The challenge

In [19]:
# Day of the challenge set, all we need to do is change this file name and run it
new_df = pd.read_csv('cleveland-test-sample.csv')

# Collapse all values 1-4 into a single value so that "num" is boolean
new_df = new_df.rename({'num':'disease','Unnamed: 0':'id'}, axis=1)
new_df['disease'] = new_df.disease.apply(lambda x: min(x, 1))

# Build a bigger data set to 
temp = pd.concat([df,new_df]).reset_index()


## chosen features

In [20]:


chosen_features = ['cp', 'trestbps', 'chol', 'restecg', 'thalach', 'exang', 'thal', 'age']
numFeatures = 5
attributes = ['age','trestbps','chol', 'thalach']
# standardizing noncatagorical data
newAttributes = []
data1 = df.copy()
for attribute in attributes:
    standard = (data1[attribute]-data1[attribute].mean())/data1[attribute].std()
    chosen_features.remove(standard.name)
    chosen_features.append(standard.name+'_s')
    standard.name = standard.name+'_s'
    newAttributes.append(standard.name)
    data1 = pd.concat([data1,standard],axis=1)
    data1.drop(attribute,axis=1,inplace=True)
featureslist = list(itertools.combinations(chosen_features,numFeatures))
featureslist = [list(x) for x in featureslist]
data1['thal'] = data1['thal'].str.replace('.0','',regex=True)


## monte carlo data split with selected features

In [21]:
# make split for one monte carlo cross validation run
def monteCarloCVSplit(data, level = 10):
    CVtest = data.sample(frac = 1/level)
    CVtrain = data.drop(CVtest.index)
    return CVtrain, CVtest

run knn on given attributes and CV split

In [22]:
def knnPredictions(CVtrain, CVTest, attributes, numOfNeighbors, target='disease'):
    nn = NearestNeighbors(n_neighbors=numOfNeighbors, metric='euclidean', algorithm='auto')
    
    # Build underlying structure with standardized data   
    X = CVtrain[attributes].values
    y = CVtrain[target].values
    testX = CVTest[attributes].values
    testy = CVTest[target].values
    display(X)
    fit = nn.fit(X)

    distances, indices = fit.kneighbors(testX, n_neighbors = numOfNeighbors)
    print(indices)
    
    # # Count the number of neighbors that have target
    # have = nbrs[nbrs[target] == 0].count()[target]
    # # Count the number of neighbors that DON'T have target
    # dontHave = nbrs[nbrs[target] == 1].count()[target]
    # # print('Have: {}\ndontHave: {}'.format(have, dontHave))
    


In [23]:
CVtrain1, CVtest1 = monteCarloCVSplit(data1)

knnPredictions(CVtrain1, CVtest1, featureslist[0] , 5)

array([[1.0, 2.0, 0.0, '6', 0.9427516619612377],
       [4.0, 2.0, 1.0, '3', 1.3838993498746968],
       [3.0, 0.0, 0.0, '3', -1.9247083094762452],
       ...,
       [4.0, 0.0, 1.0, '7', 0.2810301300910494],
       [2.0, 2.0, 0.0, '3', 0.2810301300910494],
       [3.0, 0.0, 0.0, '3', -1.8144213874978805]], dtype=object)

[[220  56 223 218  29]
 [106  79 104  36  85]
 [169 114 191 214 217]
 [122  77 225 202  44]
 [230 255  13 142  11]
 [ 44 202  50 194 122]
 [268 153 248 265  93]
 [ 65  88 134  87 176]
 [171   1 136 234  78]
 [249 128  14  81 231]
 [ 99  80 141 177 126]
 [217 191 242 114 169]
 [261 103 239  97  57]
 [  8 269 247  18  67]
 [ 18 247 269   8 221]
 [ 96 237 241 235 178]
 [262 197 183 151   5]
 [214 199  24 226 146]
 [257 259 182  73  86]
 [ 43 245  82 170  58]
 [152 160   4 192 145]
 [188   0 124 159 164]
 [ 32 168 179  25 105]
 [ 64 240 264 104  36]
 [ 14 128 249 166  21]
 [  8 269  51 247  18]
 [ 27  93  95  63 248]
 [209 156  42  25  32]
 [214 199 169 114 144]
 [258 165  40  19  72]]
